# Using the generated SDK and data models to trigger a CogShop run
NOTE: The "recommended" way of running CogShop is as follows:
1. Static model data and mappings for time-dependent data like time series represented by `ShopModel`
2. Any variations on the base configuration (like different price scenarios) represented by `ShopScenario` instances 
3. Generate a complete `ShopCase` with time series datapoints based on a `ShopScenario` instance plus `startTime` and `endTime` through the shop_trigger Cognite function.

This tutorial covers a different set-up: Triggering (Cog)Shop for a pre-generated, complete Shop "case" (as a yaml file)

## Preparation

The main entry point for the SDK is the `PowerOpsClient`

To get started, we assume you have installed the `cognite-power-ops` SDK and that all configurations have been set up in Cognite Data Fusion (CDF).

Furthermore, it is assumed that you have set up one or two `toml` files with the credentials for connecting to Cognite Data Fusion and settings for this SDK.


### Interactive Login
This only requires one `toml` file looking like this

`settings.toml`

```toml
[cognite]
  login_flow = "interactive"
  project = "<cdf-project>"
  tenant_id = "<tenant-id>"
  cdf_cluster = "<cdf-cluster>"
  client_id = "<client-id>"

[powerops]
  read_dataset = "uc:000:powerops"
  write_dataset = "uc:000:powerops"
  monitor_dataset = "uc:po:monitoring"

```

### Client Credentials Login

For this case the you can use two toml files to separate the secrets from the regular settings.


`settings.toml`

```toml
[cognite]
  login_flow = "client_credentials"
  project = "<cdf-project>"
  tenant_id = "<tenant-id>"
  cdf_cluster = "<cdf-cluster>"
  client_id = "<client-id>"

[powerops]
  read_dataset = "uc:000:powerops"
  write_dataset = "uc:000:powerops"
  monitor_dataset = "uc:po:monitoring"
```
and the `.secrets.toml`

```toml
[cognite]
  client_secret = "<client-secret>"

```

Values in `.secrets.toml` will overwrite those in `settings.toml`




In [1]:
# You can control which setting files are loaded through the environmental variable below.
# In this case, the setting files are located two levels above, in the root of the repository.
import os
from cognite.powerops._version import __version__
from cognite.powerops import PowerOpsClient


os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"

powerops = PowerOpsClient.from_settings()

WRITE_DATA_SET = powerops.datasets.write_dataset_id

print(__version__)  # Print the version of the package

0.101.1


## STEPS:
1. Upload a yaml file that can be used by Shop, and return the external id
2. Set up a ShopCase, specifying the file reference and shop version 
3. Write (upsert) the ShopCase instance to CDF, and verify that it was created using the SDK
4. Trigger a shop execution of that shop case
5. Get the resulting ShopResult


### Step 1: Upload a file 

In the case that the file already exists, this step can be skipped. 

We need the external id for constructing the `ShopCase`

In [3]:
from pathlib import Path


def upload_file(file_name: str) -> str:
    """Upload a file to CDF and return the external id of the file"""
    file_path = (Path("example_case_files") / file_name).resolve()
    file = powerops.cdf.files.upload(
        path=str(file_path),
        external_id="example_stavanger_case_file",
        name=file_name,
        data_set_id=WRITE_DATA_SET,
        mime_type="application/yaml",
        # Overwrite the file at the given external is if it already exists
        # This will also overwrite potentially existing metadata
        overwrite=True,
    )
    return file.external_id


# NB! The case in this example is minimal and just for demo purposes.
my_file_reference = upload_file(file_name="a_example_stavanger_with_commands.yaml")

### Step 2: Set up a ShopCase 


In this case, the `ShopScenario` as and its `ShopModel` are mostly superfluous. 
However, they are still added as nearly empty objects in order to set the SHOP version 

We use the generated SDK (`_generated`) as they are a direct reflection of the Data Models on CDF. Here the data classes end in `Write` since we are going to use the to send data to CDF. They all have a corresponding class used for retrieving data from CDF.

In [4]:
import datetime

from cognite.powerops.client._generated.v1.data_classes import (
    ShopCaseWrite,  # This model contains everything needed to execute a SHOP run
    ShopFileWrite,  # A container that holds a reference to a file in CDF
    ShopModelWrite,  # A static model in a file reference + references to time series data
    ShopScenarioWrite,  # A way to modify run configurations for a given model
)

# Unique identifier for the case, if not provided, an external id will be generated
EXAMPLE_CASE_EXTERNAL_ID = "example_stavanger_case_external_id"


def create_shop_case(
    file_reference: str, shop_version: str = "15.6.1.0"
) -> ShopCaseWrite:
    # a scenario is used to modify the run configuration of the model
    my_scenario = ShopScenarioWrite(
        name="dummy_scenario",
        model=ShopModelWrite(name="dummy_model", shop_version=shop_version),
    )

    shop_case_write = ShopCaseWrite(
        externalId=EXAMPLE_CASE_EXTERNAL_ID,
        # The time range SHOP is optimized over
        startTime=datetime.datetime(2024, 5, 31, 22),
        endTime=datetime.datetime(2024, 6, 2, 22),
        scenario=my_scenario,
        # The files that are used in the case
        shopFiles=[
            ShopFileWrite(
                name="stavanger_case_file",
                label="",
                fileReference=file_reference,  # external id of the file on CDF
                isAscii=False,
                order=1,
            ),
        ],
    )
    return shop_case_write


my_shop_case = create_shop_case(my_file_reference)
my_shop_case

,value
space,power_ops_instances
external_id,example_stavanger_case_external_id
data_record,{'existing_version': None}
node_type,None
scenario,"{'space': 'power_ops_instances', 'external_id'..."
start_time,2024-05-31 22:00:00
end_time,2024-06-02 22:00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


### Step 3: Upload the ShopCase

And verify that we can retrieve it too. Notice that the returned object is not suffixed with `Write`

In [5]:
def upload_shop_case(shop_case: ShopCaseWrite) -> None:
    """Upload a shop case to CDF"""
    return powerops.v1.upsert(shop_case)


upload_shop_case(my_shop_case)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='example_stavanger_case_external_id', version=1) at 0x7f2428227450>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:5c364e4010514e0e905446f0cebff4db', version=1) at 0x7f23d4b2ec10>, <NodeApplyResult(space='power_ops_instances', external_id='shopscenario:bc42f5f1180243ce80efb9e020ce740f', version=1) at 0x7f23d4b2ea90>, <NodeApplyResult(space='power_ops_instances', external_id='shopmodel:ac1533af987b4b6aa9261cfa19df4692', version=1) at 0x7f23d4b2d990>], edges=[<EdgeApplyResult(space='power_ops_instances', external_id='example_stavanger_case_external_id:shopfile:5c364e4010514e0e905446f0cebff4db', version=1) at 0x7f23d4b2cb90>], time_series=[])

In [6]:
from cognite.powerops.client._generated.v1.data_classes._shop_case import ShopCase


# NB! This step is not necessary, if the upsert was successful we know the case is in CDF
def retrieve_shop_case(case_external_id: str) -> ShopCase:
    """Retrieve a shop case from CDF"""
    # return powerops.v1.get(external_id)
    return powerops.v1.day_ahead_bid.shop_case.retrieve(external_id=case_external_id)


retrieved_shop_case = retrieve_shop_case(EXAMPLE_CASE_EXTERNAL_ID)
retrieved_shop_case

,value
space,power_ops_instances
external_id,example_stavanger_case_external_id
data_record,"{'version': 1, 'last_updated_time': 2024-10-21..."
node_type,None
scenario,shopscenario:bc42f5f1180243ce80efb9e020ce740f
start_time,2024-05-31 22:00:00+00:00
end_time,2024-06-02 22:00:00+00:00
shop_files,[shopfile:5c364e4010514e0e905446f0cebff4db]


### Step 4: Trigger the case

We will here directly use the new `trigger_shop_case` method on the `powerops.cogshop` client

In [7]:
powerops.cogshop.trigger_shop_case(EXAMPLE_CASE_EXTERNAL_ID)

### Step 5: View the ShopResult generated 

Explanations for (some) of the fields of the `ShopResult`: 

* External id: The external id of the `ShopResult`
* Case: The external id of the `ShopCase` that the instance belongs to
* Objective value: The objective value of the Shop execution
* Pre run: A file reference to the pre run yaml file
* Post run: A file reference to the post run yaml file
* Messages: A file reference to the logs generated by Shop
* Cplex logs: A file reference to the cplex logs generated by Shop
* Data record: The data record of the instance, contains `last_updated_time` and `created_time`



#### 1. Querying for ShopResult instance, given their ShopCase external id

Note that it may take a while to run (Cog)Shop. 
If the run is not completed then no results are returned

In [8]:
from cognite.powerops.client._generated.v1.data_classes import ShopResultList

# Query the shop results based on the external id of the shop case
# Since it is possible to have multiple results for a case (or view results of multiple cases)
# the return type is a list
def result_instance_query(shop_case_external_id: str) -> ShopResultList:
    # Generate the query -- This can be more complex if needed, see tutorial `advanced_cogshop_run`
    call_query = powerops.v1.day_ahead_bid.shop_result(case=shop_case_external_id)
    return call_query.query()

In [10]:
# We convert the result to a pandas dataframe for easier data handling
all_results_df = result_instance_query(EXAMPLE_CASE_EXTERNAL_ID).to_pandas()
all_results_df

,space,external_id,case,objective_value,pre_run,post_run,messages,cplex_logs,alerts,output_time_series,node_type,data_record
0,power_ops_instances,shop_result__89893a72-e7f9-40a3-8ac9-93fac49eb9e6,example_stavanger_case_external_id,"{'total': -380036.7316799998, 'load_value': 0....",example_stavanger_case_file,POWEROPS_SHOP_post-run-89893a72-e7f9-40a3-8ac9...,POWEROPS_SHOP_shop-89893a72-e7f9-40a3-8ac9-93f...,POWEROPS_SHOP_cplex-89893a72-e7f9-40a3-8ac9-93...,None,None,None,"{'version': 1, 'last_updated_time': 2024-10-21..."


#### 2. Using the external ID of a ShopResult to retrieve and inspect it  

In [11]:
import pandas as pd
from cognite.powerops.client._generated.v1.data_classes import (
    ShopResult,
    ShopResultList,
)


# Retrieve a specific shop result instance based on the external id of the shop result
def result_instance_retrieve(external_id: str | list[str]) -> ShopResultList:
    """Returns a list of ShopResult instances based on the external id(s) of the shop result"""
    instance = powerops.v1.shop_based_day_ahead_bid_process.shop_result.retrieve(
        external_id=external_id
    )
    return instance

In [12]:
first_result = all_results_df.head(1)

# Retrieve the first result instance from the dataframe above. This guarantees that the result exists
shop_result: ShopResult = result_instance_retrieve(first_result.external_id)[0]
shop_result

,value
space,power_ops_instances
external_id,shop_result__89893a72-e7f9-40a3-8ac9-93fac49eb9e6
data_record,"{'version': 1, 'last_updated_time': 2024-10-21..."
node_type,None
case,example_stavanger_case_external_id
objective_value,"{'total': -380036.7316799998, 'load_value': 0...."
pre_run,example_stavanger_case_file
post_run,POWEROPS_SHOP_post-run-89893a72-e7f9-40a3-8ac9...
messages,POWEROPS_SHOP_shop-89893a72-e7f9-40a3-8ac9-93f...
cplex_logs,POWEROPS_SHOP_cplex-89893a72-e7f9-40a3-8ac9-93...


## Cleaning up the instances that were created

Instances can be deleted by using the powerops client and the `external_id`s of everything we have created 

In [13]:
all_external_ids = all_results_df.external_id.tolist()
all_external_ids.append(EXAMPLE_CASE_EXTERNAL_ID)

all_external_ids

['shop_result__89893a72-e7f9-40a3-8ac9-93fac49eb9e6',
 'example_stavanger_case_external_id']

In [14]:
powerops.v1.delete(external_id=all_external_ids)


InstancesDeleteResult(nodes=[NodeId(space='power_ops_instances', external_id='shop_result__89893a72-e7f9-40a3-8ac9-93fac49eb9e6'), NodeId(space='power_ops_instances', external_id='example_stavanger_case_external_id')], edges=[])